# 模式识别实验二：神经网络算法

In [1]:
import numpy as np
#导入库文件
from sklearn import datasets
from sklearn.model_selection import train_test_split

# 定义神经网络类及其函数

In [2]:
#定义sigmoid函数
def sigmoid(data):
    return 1.0 /(1.0 + np.exp(-data))    

In [3]:
#定义sigmoid函数的导数
def derive_sigmoid(data):
    return sigmoid(data)*(1 - sigmoid(data))

In [42]:
class BPNN:
    #定义初始化函数
    #n_input,n_nodes,n_output分别为输入层、隐层、输出层节点个数
    def init_function(self,n_input,n_nodes,n_output):
        self.n_output = n_output
        self.value1 = np.random.random((n_nodes,n_input+1))*0.1
        self.value2 = np.random.random((self.n_output,n_nodes+1))*0.1
        self.hiddenlayer_input = np.zeros((n_nodes,1))
        self.outputlayer_input = np.zeros((n_output,1))
        self.predict = np.zeros((self.n_output,1))
        self.error = self.predict
    
    def forward_function(self,data):
        self.hiddenlayer_input = self.value1 @ data
        hiddenlayer_output = np.vstack((sigmoid(self.hiddenlayer_input),[1]))
        
        self.outputlayer_input = self.value2 @ hiddenlayer_output
        self.predict = sigmoid(self.outputlayer_input)
        
    def backward_function(self,data,label,alpha):
        self.error = self.predict - label
        
        sigma2 = self.error * derive_sigmoid(self.outputlayer_input)
        sigma1 = np.transpose(self.value2[:,:-1]) @ sigma2 * derive_sigmoid(self.hiddenlayer_input)
    
        delta2 = sigma2 @ np.vstack((sigmoid(self.hiddenlayer_input),[1])).T
        
        delta1 = sigma1 @ data.T
        
        self.value2 = self.value2 - delta2 * alpha 
        self.value1 = self.value1 - delta1 * alpha 
        
    def train_function(self,dataset,labels,alpha,iteration):
        [length,width] = dataset.shape
        
        for i in range(iteration):
            indice = np.random.randint(length)
            data = dataset[indice,:].reshape((width,1))
            label = labels[indice,:].reshape((self.n_output,1))
            self.forward_function(data)
            self.backward_function(data,label,alpha)
        print('training process finished')
        
    def predict_function(self,dataset):
        [length,width] = dataset.shape
        predict = np.zeros((length,self.n_output))
        
        for i in range(length):
            data = dataset[i,:].reshape((width,1))
            self.forward_function(data)
            indice = np.argmax(self.predict)
            predict[i,indice] = 1
        return predict
    
    def loss_function(self,predict,labels):
        errors = predict - labels
        errors = np.sum(abs(errors),1)/2 
        error_rate = np.mean(errors)
        print('error_rate: ',error_rate)
        return errors
        
    def pretrait_function(self,dataset,pre_labels):
        [length,width] = dataset.shape
        dataset = np.hstack((dataset,np.ones((length,1))))
        
        labels = np.zeros((length,self.n_output))
        for i in range(length):
            indice = int(pre_labels[i])
            labels[i,indice] = 1
        return dataset,labels

## BP算法 异或问题

In [43]:
data = np.array([[0,0],[0,1],[1,1],[1,0]]).reshape(4,2)
label = np.array([[0],[1],[0],[1]])
dataset1 = (np.random.random((50,2)) - 0.5) * 0.2 + data[0,:]
dataset2 = (np.random.random((50,2)) - 0.5) * 0.2 + data[1,:]
dataset3 = (np.random.random((50,2)) - 0.5) * 0.2 + data[2,:]
dataset4 = (np.random.random((50,2)) - 0.5) * 0.2 + data[3,:]
pre_dataset = np.vstack((dataset1,dataset2,dataset3,dataset4))

pre_labels = np.zeros((200,1))
pre_labels[50:100] = 1
pre_labels[150:200] = 1

print('dataset size: ', pre_dataset.shape)
print('labels size: ',pre_labels.shape)

dataset size:  (200, 2)
labels size:  (200, 1)


In [53]:
bpnn = BPNN()
bpnn.init_function(2,5,2)
[dataset, labels] = bpnn.pretrait_function(pre_dataset,pre_labels)
X_train,X_test,y_train,y_test = train_test_split(dataset, labels, test_size = 0.2, random_state = 0)
bpnn.train_function(X_train,y_train,alpha=1,iteration=5000)
predict = bpnn.predict_function(X_test)
errors = bpnn.loss_function(predict,y_test)
print(errors)

training process finished
error_rate:  0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## BP算法 Iris数据集

In [45]:
#加载数据并显示
iris=datasets.load_iris()

#data对应了样本的4个特征，150行4列
print(iris.data.shape) 

#target对应样本的标签，150行1列
print(iris.target.shape)

(150, 4)
(150,)


In [48]:
iris_bpnn = BPNN()
iris_bpnn.init_function(4,10,3)
[dataset, labels] = iris_bpnn.pretrait_function(iris.data,iris.target)
X_train,X_test,y_train,y_test = train_test_split(dataset, labels, test_size = 0.2, random_state = 0)
iris_bpnn.train_function(X_train,y_train,alpha=0.5,iteration=5000)
predict = iris_bpnn.predict_function(X_test)
errors = iris_bpnn.loss_function(predict,y_test)
print(errors)

training process finished
error_rate:  0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
